## In-Memory Persistence and Memory Management

Spark's performance advantage over MapReduce is greatest in use cases involving repeated computations. Much of this performcance increase is due to Spark's use of in-memory persistence. Rather than writing to disk between each pass through the data, Spark has the option of keeping the data on the executors loaded into memory. That way, the data on each partition is available in-memory each time it needs to be accessed. 

Spark offers three options for memory management: in-memory as deserialized data, in-memory as serialized data, and on disk. Each has different space and time advantages: 

In memory as deserialized Java objects
The most intuitive way to store objects in RDDs is as the original deserialized Java objects that are defined by the driver program. This form of in-memory is the fatest, since it reduces serialization time; however, it may not be the most memory efficient, since it requires the data to be stored as objects.

As serialized data

Using the standard Java serialization library, Spark objects are converted into streams of bytes as they are moved around the network. This approach may be slower, since serialized data is more CPU-intensive to read than deserialized data; however, it is often more memory efficient, since it allows the user to choose a more efficient representation. While Java serialization is more efficient than full objects, Kryo serialization can be even more space efficient.

On disk

RDDs, whose partitions are too large to be stored in RAM on each of the executors, can be written to disk. This strategy is obviously slower for repeated computations, but can be more fault-tolerant for long sequences of transformations, and may be the only feasible option for enormous computations.

The persist() function in the RDD class lets the user control how the RDD is stored. By default, persist() stores an RDD as deserialized objects in memory, but the user can pass one of numerous storage options to the persist() function to control how th RDD is stored. We will cover the different options for RDD reuse in "Types of Reuse: Cache, Persist, Checkpoint, Shuffle Files" on page 116. When persisting RDDs, the default implementation of RDDs evicts the least recently used partition (called LRU caching) if the space it takes is required to compute or to cache a new partition. However, you can change this behavior and control Spark's memory prioritization with the persistencePriority() function in the RDD class.

## Immutability and the RDD Interface

Spark defines an RDD interface with the properties that each type of RDD must implement. These properties include the RDD's dependencies and information about data locality that are needed for the execution engine to compute that RDD. Since RDDs are statically typed and immutable, calling a transformation on one RDD will not modify the original RDD but rather return a new RDD object with a new definition of the RDD's properties.

RDDs can be created in three ways: (1) by transforming an existing RDD; (2) from a SparkContext, which is the API's gateway to Spark for your application; and (3) converting a DataFrame or Dataset. The SparkContext represents the connection between a Spark cluster and one running Spark application. The SparkContext can be used to create an RDD from a local Scala object(using the makeRDD or parallelize methods) or by reading from stable storage (text files, binary files, a Hadoop Context, or a Hadoop File). DataFrames and Datasets can be read using the Spark SQL equivalent to a SparkContext, the SparkSession.

Internally, Spark uses five main properties to represent an RDD. The three required properties are the list of parittion objects that make up the RDD, a function for computing an iterator of each partition, and a list of dependencies on other RDDs. Optionally, RDDs also include a partitioner (for RDDs of rows of key/value pairs represented as Scala tuples) and a list of preferred locations (for the HDFS file). As an end user, you will rarely need these five properties and are more likely to use predefined RDD transformations. However, it is helpful to understand the properties and know how to access them for debuggin and for a better conceptual understanding. These five properties correspond to the following five methods available to the end user.

partitions()

Returns an array of the partition objects that make up the parts of the distributed dataset. In the case of an RDD with a partitioner, te value of the index of each partition will correspond to the value of the getPArititon function for each key in the data associated with that partition.

iterator(p, parentIters)

Computes the elements of partition p given iterators for each of its parent paritions.

depenencies()
Returns a sequence of dependency objects. The dependencis let the scheduler know how this RDD depends on other RDDs. There are two kinds of dependencies: narrow dependencies, which represent partitions that depend on one or a small subset of partitions in the parent, and wide dependencies, which are used when a partition can only be computed by rearrangin all the data in the parent. We will discuss the types of dependenices in 

partitioner()
Returns a Scala option type of a partitioner object if the RDD has a function between element and partition associated with it, such as a hashPartitioner. This function returns None for all RDDs that are not of type tuple. An RDD that represents an HDFS file has a partition for each block of the file. We will discuss partitioning in detail in 

preferredLocations(p)
Returns information about the data locality of a partition, p. Specifically, this function returns a sequence of strings representing some information about each of the nodes where the split p is stored. In an RDD representing an HDFS file, each string in the result of preferredLocations is the Hadoop name of the node where that partition is stored.

# Types of RDDs

The implementation of the Spark Scala API contains an abstract class, RDD, which contains not oonly the five core functions of RDDs, but also those transformations and actions that are available to all RDDs, such as map and collect. Functions defined only on RDDs of a particular type are defined in several RDD function classes, including PairRDDFunctions, OrderedRDDFunctions, and GroupedRDDFunctions. The additional methods in these classes are made available by implicit conversion from the abstract RDD class, based on type information on when a transformation is applied to an RDD.

The Spark API also contains implementations of the RDD class that define more specific behavior by overriding the core properties of the RDD. These include the NewHadoopRDD class discussed previously- which represents an RDD created from an HDFS filesystem - and ShuffledRDD, which represents an RDD that was already partitioned. Each of these RDD iplementations contains funcionality that is specific to RDDs of that type. Creating an RDD, either through a transformation or from a SparkContext, will return one of these implementations of the RDD class. Some RDD operations have a different signature in Java than in Scala. These are defined in the JavaRDD.java class.

# Functions on RDDs: Transformations Versus Actions

There are two types of functions defeind on RDDs: actions and transformations. Actions are functions that return something that is not an RDD, including a side effect, and transformations are functions that return another RDD.

Each Spark program must contain an action, since actions either bring information back to the driver or write the data to stable storage. Actions are what force evaluation of a Spark program. Persist calls also force evaluation, but usually do not mark the end of Spark job. Actions that bring data back to the driver include collect, count, collectAsMap, sample, reduce, and take.

Actions that write to storage include saveAsTextFile, saveAsSequenceFile, and saveAsObjectFile. Most actions that save to Hadoop are made available only on RDDs of key/value pairs; they are defined both in the PairRDDFunctions class(which provides methods for RDDs of uple type by implicit conversion) and the NewHadoopRDD class, which is an implementation for RDDs that were created by reading from Hadoop. Some saving functions, like saveAsTextFile and saveAsObjectFile, are available on all RDDs, and they work by adding an implicit null key to each record(which is then ignored by the saving level). Functions that erturn nothing, such as foreach, are also actions: they force execution of a Spark job. foreach can be used to force evaluation of an RDD, but is also often used to write out to nonsupported formats.

# Wide Versus Narrow Dependencies

For the purpose of understanding how RDDs are evaluate, the most important thing to know about transformations is that they fail into two categories: transformations with narrow dependencies and transformations with wide dependencies. The narrow versus wide distinction has significant implications for the way Spark evaluates a transformation and, consequently, for its performance. We will define narrow and wide transformations for the purpose of understanding Spark's execution paradigm in "Spark job scheduling" on page 19 of this chapter, but we will save the longer explanation of the performance considerations associated with them for Chapter 5.

Conceptually, narrow transformations are those in which each partition in the child RDD has simple, finite dependencies on partitions in the parent RDD. Dependencies are only narrow if they can be determined at design time, irrespectie of the values of the records in the parent partitions, and if each parent has at most one child partition. Specifically, partitions in narrow transformations can either depend on one parent(such as in the map operator), or a unique subset of the parent partitions that is known at design time. Thus narrow transformations can be executed on an arbitrary subset of the data without any information about the other partitions. In contrast, transformations with wide dependencies cannot be executed on arbitrary rows and instead require the data to be partitioned in a particular way, e.g., according the value of their key. In sort, for example, records have to be partitioned so that keys in the same range are on the same partiton. Transformations with wide dependencies include sort, reduceByKey, groupByKey, join, and anything that calls the rePartition function.

In certain instances, for example, when Spark already knows the data is partitioned in a certain way, operations with wide dependencies do not cause a shuffle. If an operation will require a shuffle to be executed, Spark adds a ShuffleDependency object to the dependency list associated with the RDD. In general, shuffles are expensive. They become more expensive with more data and when a greater protportion of that data has to be moved to a new partition during the shuffle. As we will discuss at length in Chaprter 6, we can get a lot of performance gain out of Spark programs by doing fewer and less expensive shuffles.

The next two diagrams illustrate the difference in the dependency graph for transformations with narrow dependencies versus transformations with wide dependencies. shows narrow dependencies in which each child partition (each of the blue squares on the bottom rows) depends on a known subset of parent partitions. Narrow dependencies are shown with blue arrows. The left represents a dependency graph of narrow transformations (such as map, filter, mapPartitions, and flatMap). On the upper right are dependencies between partitions for coalesce, a narrow transformation. In this instance we try to illustrate that a transformation can still qualify as narrow if the child partitions may depend on multiple parent partitions, so long as the set of parent parititons can be determined regardless of the values of the data in the partitions.

# Spark Job Scheduling

A Spark applciation consists of a driver process, which is where the high-level Spark logic is written, and a series of executor processes that can be scattered across the nodes of a cluster. The Spark program itself runs in the driver node and sends some instructions to the executors. One Spark cluster can run several Spark applications concurrently. The applications are scheduled by the cluster manager and correspond to one SparkContext. Spark applications can, in turn multiple concurrent jobs. Jobs correspond to each action aclled on an RDD in a given application. In this section, we will describe the Spark application and how it launches Spark jobs: the processes that compute RDD transformations.

## Resource Allocation Across Applications

Spark offers two ways of allocating resources across applications: static, dynamic

## The Spark Application

A Spark application corresponds to a set of Spark jobs defined by one SparkContet in the driver program. A Spark appcalication begins when a SparkContext is started. When the SparkContext is started, a driver and a series of executors are started on the worker nodes of the cluster. Each executor is its own Java Virtual Machine, and an executor cannot span multiple nodes although one node may contain several executors.

The SparkContet determines how many resources are allotted to each executor. When a Spark job is launched, each executor has slots for running the tasks needed to compute an RDD. In this way, we can think of one SparkContext as one set of configuration parameters for running Spark jobs. These parameters are exposded in the SparkConf object, which is used to create SparkContext. We will discuss how to use the paramters in Appendix A. Applications often, but not always correspond to users. That is, each Spark program running on your cluster likely uses one SparkContext.

### Default Spark Scheduler

By default, Spark schedules jobs on a first in, first ou basis. However, Spark does offer a fair scheduler, which assigns tasks to concurrent jobs in round-robin fachion, i.e., parceling out a few tasks for each job until the bjos are all complete. The fair scheduler ensures that jobs get a more even share of cluster resources. The Spark application then launches jobs in the order that their corresponding actions were called on the SparkContext.

# The Anatomy of a Spark Job

In the spark lazy evaluation paradigm, a Spark application doesn't do anything until the driver program calls an action. With each action, the Spark scheduler builds an execution graph and launches a Spark job. Each job consists of stages, which are steps in the transformation of the data needed to materialize the final RDD. Each stage consists of a collection of tasks that represent each parallel computation and are performed on the executors.

## Jobs

A Spark job is the highest element of Spark's execution hierarchy. Each Spark job corresponds to one action, and each action is called by the driver program of a Spark application. As we discussed in "Functions on RDDs: Transformations Versus Actions" on page 17, one way to conceptualize an action is as something that brings data out of the RDD world of Spark into some other storage system.

The edges of the Spark execution graph are based on dependencies between the partitions in RDD transformations. Thus, an operation that returns something other than RDD cannot have any children. In graph theory, we would say the action forms a "leaf" in the DAG. Thus, an arbitrarily large set of transformations may be associated with one execution graph. However, an soon as an action is called, Spark can no longer add to that graph. The application launches a job including those transformations that were needed to evaluate the final RDD that called the action.

## Stages

Recall that Spark lazily evaluates transformations; transformations are not executes until an action is called. As mentioned previously, a job is defined by calling an action. The action may include one or several transformations, and wide tranformations define the breakdown of jobs into stages.

Each stage corresponds to a shuffle dependency created by a wide transformation in the Spark program. At a high level, one stage can be thought of as the set of computations that can each be computed on one executor without communication with other executors or with the driver. In other words, a new stage begins whenever netwrok communication between workers is required; for instance, in a shuffle.

These dependencies that create stage boundaries are called ShuffleDependencies. As we discussed in "Wide", shuffles are caused by those wide transformations, such as sort or groupByKey, which require the data to be redistributed across the partitions. Several transformations with narrow dependencies can be grouped into one stage.

However, the wide transformations needed to compute one RDD have to be computed in sequence. Thus it is usually desirable to design your program to require fewer shuffles.

## Tasks

A stage consists of tasks. The task is the smallest unit in the execution hierarchy, and each can represent one local computation. All of the tasks in one stage execute the same code on a different piece of the data. One task cannot be executed on more than one executor. However, each executor has a dynamically allocated number of slots for running.

A cluster cannot necessarily run every task in parallel for each stage. Each executor has a number of cores. The number of cores per executor is configured at the application level, but likely corresponding to the physical cores on a cluter. Spark can run no more tasks at once than the total number of executor cores allocated for the application. We can calculate the number of tasks from the settings from the Spark Conf as (total number of executor cores = # of cores per executor x number of executors). If there are more parititions (and thus more tasks) than the number of slots for running tasks, then the extra tasks will be allocated to the executors as the first round of tasks finish and resources are available. In most cases, all the tasks for one stage must be completed before the next stage can start. The process of distributing these tasks is done by the TaskScheduler and varies depending on whether the fair scheduler or FIFO scheuler is used.

# DataFramse, Datasets, and Spark SQL

Like RDDs, DAtaFrames and Datasets represent distributed collections, with additional schema information not found in RDDs. This additional schema information is used to provide a more efficient storage layer, and in the optimizer to perform additional optimizations. Beyond schema information, the operations performed on Datasets and DataFrames are such that the optimizer can inspect the logical meaning rather than arbitrary functions. DataFrammes are Datasets of a special Row object, which doesn't provide any compile-time type checking. The strongly typed Dataset API shines especially for use with more RDD-like functional operations. Compared to working with RDDs, DataFrames allow Spark's optimizer to better understand our code and our data, which allows for a new class of optimizations we explore in "Query Optimizer" on page 69.

## Getting Started with the SparkSEssion (or HiveContext or SQLContext)

Much as the SparkContext is the entry point for all Spark applications, and the StreamingContext is for all streaming applications, the SparkSession serves as the entry point for Spark SQL. Like with all of the Spark components, you need to import a few extra components as shown in Example 3-1/

SparkSession is generally created using the builder pattern, along with getOrCreate(), which will return an existing session if one is already running. The builder can take string-based configuration keys config(key, value), and shortcuts exist for a number of common params. One of the more important shortcuts is enableHiveSupport(), which will give you access to Hive UDFs and does not require a Hive installation - but does require certain extra JARs. 

Before Spark 2.0, instead of the SparkSEssion, two separate entry points were used for Spark SQL. The names of these entry points can be a bit confusing, and it is important to note the HiveContext does not require a Hive installation. The primary reason to use the SQLContext is if you have conflicts with the Hive dependencies that cannot be resolved. The HiveContext has a more complete SQL parser compared to the SQLContext as well as additional user-defined functions. Example 3-4 shows how to create a legacy HiveContext. the SparkSEssion should be preferred when possible, followed by the HiveContext, then SQLContext. Not all libraires, or even all Spark code, has been updated to take the SparkSession and in some cases you will find functions that still expect s SQLContext or HiveContext.

If you need to contstruct one of the legay interfaces (SQLContext or HiveContext) the additional imports in Example 3-3 will be useful.

## Spark SQL Dependencies

Like the other components in Spark, using Spark SQL requires adding additional dependencies. If you have conflicts with the Hive JARs you can't fix through shading, you can just limit yourself to the spark-sql JAR-although you want to have access to the Hive dependencies without also inclusing the spark-hive JAr.

To enable Hive support in SparkSession or use the HiveContext you will need to add both Spark's SQL and Hive components to your dependencies.

## Managin Spark Dependencies

While managing these dependencies by hand isn't particularly challenging, sometimes mistakes can be made when updating versions. The sbt-spark-package plug-in can simplify managing Spark dependencies. This plug-in is normally used for creating community packages, but als assist in building software that depends on Spark. To add the plug-in to your sbt build you need to create a project/plugins.sbt file and make sure it contains the code in Example 3-7.

## Basics of Schemas

The schema information, and the optimizations it enables, is one of the core differences between Spark SQL and core Spark. Inspecting the schema is especially useful for DataFrames since you don't have the templated type you do with RDDs or Datasets. Schemas are normally handled automatically by Spark SQL, either inferred when loading the data or computed based on the parent DataFrames and the transformation being applied.

## DataFrame API

Spark SQL's DataFrame API allows us to work with DataFrames without having to register temporary tables or generate SQL expressions. The DataFrame API has both transformations and actions. THe transformations on DataFrames are more relational in nature, with the Dataset API offering a more functional-style API.

### Transformations

Transformations on DataFrames are similar in concept to RDD transformations, but with a more relational flavor. Instead of specifying arbitrary functions, which the optimizer is unable to introspect, you use a restricted expression syntax so the optimizer can have more information. As with RDDs, we can broadly break down transformations into simple single DataFrame, multiple DataFrame, key/value, and grouped/windowed transformations.

### Simple DataFrame transformations and SQL expressions

Simple DataFrame transformations allow us to do most of the standard things one can do when working a row at a time. You can still do many of the same operations defined on RDDs, except using Spark SQL expressions instead of arbitrary functions. To illustrate this we will start by examining the different kinds of filter operations available on DataFrames.

DataFrame functions, like filter, accept Spark SQL expressions instead of lambdas. These expressions allow the optimizer to understand what the condition represents, and with filter, it can often be used to skip reading unnecessary records.

To get started, let's look at a SQL expression to filter our data for unhappy pandas using our existing schema. The first step is looking up the column that contains this informatino. In our case it is happy, and for our DataFrame we access the column through the apply function. The filter expression requires the expression to return a boolean value, and if you wanted to select happy pandas, the entire expression could be retrieving the column value. However, since we want to find the unhappy pandas, we can check to see that happy isn't true using the !== opertor as shown in Example 3-18.

This illustrates how to access a specific column from a DataFrame. For accessing other structures inside of DataFrames, like nested structs, keyed maps, and array elements, use the same apply syntax. So, if the first element in the attributes array represent squishiness, and you only want very squishy pandas, you can access that element by writing df

Spark SQL's DataFrame API has a very large set of operators available. You can use all of the standard mathematical operators on floating points, along with the standard logical and bitwise opertions. Columns use === and !== for equality to avoid conflict with Scala internals. For columns of strings, startsWith/endsWith, substr, like and isNull are all available. The full set of operations is listed in.

### Beyond row-by-row transformations

Sometimes applying a row-by-row decision, as you can with filter, isn't enough. Spark SQL also allows us to select the unique rows by calling dropDuplicates, but as with the similar operation on RDDs, this can require a shuffle, so is often much slower than filter. Unlike with RDDs, dropDuplicates can optionally drop rows based on only a subset of the columns, such as ID field, as shown in Example 3-22.

### Aggregates and groupBy

Spark SQL has many powerful aggregates, and thanks to its optimizer it can be easy to combine many aggregates into one single action/query. Like with Pandas's DataFrams, groupBy returns special objects on which we can ask for certain aggregations to be perormed. In pre-2.0 versions of Spark, this was a generic GroupedData, but in versions 2.0 and beyond, DataFrams groupBy is the same as one Datasets.

Aggregations on Datasets have extra functionality, returning a GroupedDataset or a KeyValueGroupedDataset when grouped with an arbitrary function, and a RelationalGroupedDataset wen grouped with a relational DatasetDSI expression. The additional typed functionality is discussed in Grouped Operations on Datasets on page 65, and the common untyped DataFrame and Dataset groupBy functionality is explored here.

## Multi-DataFrame Transformations

Beyond single DataFrame transformations you can perform operations that depend on multiple DataFrames. The ones that first pop into our heads are most likely the different types of joins, which are covered in Chapter 4, but beyond that you can also perform a number of set-like operations between DataFrames.

### Set-like operations

The DataFrame set-like operations allow us to perform many operations that are most commonly thought of as set operations. These operations behave a bit differently than traditional set operations since we don't have the restriction of unique elements.

While you are likely already familiar with the results of set-like operations from regular Spark and Learning Spark, it's important to review the cost of these operations in Table 3-9.

## Plain Old SQL Queries and Interacting with Hive Data

Sometimes, it's better to use regular SQL queries instead of building up our operations on DataFrames. If you are connected to a Hive Metastore we can directly write SQL queries against the Hive tables and get the results as a DataFrame. If you have a DataFrae you want to write SQL queries against, you can register it as a temporary table, as shown in Example 3-30. Datasets can also be converted back to DataFrames and registered for querying against.

## Data Loading and Saving Functions

Spark SQL has a different way of loading and saving data than core Spark. To be able to push down certaintypes of operations to the storage layer, Spark SQL has its own Data Source API. Data sources are able to specify and control which type of operations should be pushed down to the data source. As developers, you don'y need to worry too much about the internal activity going on here, unless the data sources you are looking for are not supported.

## DataFrameWriter and DataFrameReader

The DataFrameWriter and the DataFrameReader cover writing and reading from external data sources The DataFrameWriter is accessed by calling write on a DataFrame or Dataset. The dataFrameReader can be accessed through read on a SQLContext.

## Formats

When reading or writing you specify the format by calling format on the DataFrameWirtier/DataFrameReader. Format-specific parameters, such as number of records to be sampled for JSON, are specified by either providing a map of options with or setting option-by-option with option on the reader/writer.

### JSON

Loading and writing JSON is supported directly in Spark SQL, and despite the lack of schema information in JSON, Spark SQL is able to infer a shcema for us by sampling the records. Loading JSON data is more expensive than loading many data sources, since Spark needs to read some of the records to determine the schema information. If the schema between records varies widely, you can increase the percentage of records read to determine the schema by setting sammplingRatio to a higher value, as in Example 3-33 where we set the sample ratio to 100%.

Since our input may contain some invalid JSON records we may wish to filter out, we can also take in as RDD of strings. This allow us to load the input as a standard text file, filter out our invalid records, and then load the data into JSON. This is done by using the built-in json function on the DataFrameReader, which takes RDDs or paths and is shown in Example 3-34. Methods for converting RDDs of regular objects are covered in RDDs on page 56.

### JDBC

The JDBC data source represents a natural Spark SQL data source, one that supports many of the same operations. Since different database vendors have slightly differnt JDBC implementations, you need to add the HAR for your JDBC data sources. Since SQL field types vary as well, Spark uses JdbcDialects with built-in dialects for DB2, Derby, MsSQL, MySQL, Oracle, and Postges.

While Spark supports many different JDBC sources, it does not ship with the JARs required to talk to all of these databases. If you are submitting your Spark job with spark-submit you can download the required JARs to the host you are launching and include them by specifying --hars or supply the Maven coordinates to --packages. Since the Spark Shell is also launched this way, the same syntax works and you can use it to include the MySQL JDBC JAR in Example 3-35.

As with the other built-in data source, there exists a convenience wrapper for speciying the properties requried to load JDBC data, illustrated in Example 3-36. The convenience wrapper JDBC accepts the URL, table, and a java.util.Properties object for connection properties. The properties object is merged with the properties that are set on the reader/writer itself. While the properties object is required, an empty properties object can be provided and properties instead specified on the reader/writer.

The API for saving a DataFrame is very similar to the API used for loading. The save() function needs no path since the information is already specified, as illustrated in Example 3-37, just as with loading.

### Parquet

Apache Parquet files are a common format directly supported in Spark SQL, and they are incredibly space-efficient and popular. Apache Parquet's popularity comes from a number of features, inclusing the ability to easily split across multiple files, compression, nested types, and many others discussed in the Parquet documentation. Since Parquet is such a popular format, there are some additional options available in Spark for the reading and writing of Parquet files. These options are listed in Table 3-10. Unlike third-party data sources, these options are mostly configured on the SQLContex, although some can be configured on either the SQLContext or DataFRmaeReader/Writer.

### Hive tables

Interacting with Hive tables adds another option beyond the other formats. As converedin Plain Old SQL Queries and Interacting with Hive Data on page 49, one option for bringin in data from a Hive table is writing a SQL query against it and having the result as a DataFrame. The DataFrame's reader and writer interfaces can also be used with Hive tables, as with the rest of the data sources, as illustrated in Example 3-40.

### RDDs

Spark SQL DataFrames can easily be converted to RDDs of Row objects, and can also be created from RDDs of Row objects as well as JavaBeans, Scala case classes, and tuples. For RDDs of strings in JSON format, you can use the methods discussed in "JSON" on page 52. Datasets of type T can also easily be converted to RDDs of type T, which can provide a useful bridge for DataFrames to RDDs of concrete case classes instead of Row objects. RDDs are a special-case data source, since when going to/from RDDs, the data remains inside of Spark without writing out to or reading from an external system.

When you create a DataFrame from an RDD, Spark SQL needs to add schema information. If you are creating the DataFrame from an RDD of case classes or plain olld Java objects, Spark SQL is able to use reflection to automatically determine the schema, as shown in Example 3-42. You can also manually specify the schema for your data using the structure discussed in "Basics of Schemas" on page 33. This can be especially useful if some of your fields are not nullable. You must specify the schema yourself if Spark SQL is unable to determine the schema through reflection, such as an RDD of Row objects.

Since a row can contain anything, you need to specify the type as you fetch the values for each column in the row. With Datasets you can directly get back an RDD templated on the same type, which can make the conversion back to a useful RDD much simpler.

### Local collections

Much like with RDDs, you can also create DataFrames from local collections and bring them back as local collections, as illustrated in Example 3-44 The same memory requirements apply; namely, the entire contents of the DataFrame will be in-memory in the driver program. As such, distributing local collections is normally limited to unit tests, or joining small datasets with larger distributed datasets.

Collecting data back as a local collection is more common and often done post aggregations or filtering on the data. For example, with ML pipelines collecting the coefficients or collecting the quantiles to the driver. Example 3-45 shows how to collect a DataFrame back locally. For larger datasets, saving to an external storage system is recommended.

### Additional formats

As with core Spark, the data from formats that ship directly with Spark only begin to scratch the surface of the types of systems with which you can interact. Some vendors publish their own implementations, and many are published on Spark Packages. As of thes writing there are over twenty formats listed on the Data Source's page with the most popular being Avro, Redshift, CSV, and a unified wrapper around 6+ databases called deep-spark.

Spark packages can be included in your application in a few different ways. During the exploration phase you can include them by specifying packages on the commnad line, as in Example 3-46. The same approach can be usedwhen submitting your application with spark-submit, but this only includes the package at runtime, not at compile time. For including at compile time you can add the Maven coordinates to your builds, or, if building with sbt, the sbt-spark-package plug-in simplifies package dependencies with spDependencies. Otherwise, manually listing them as in Example 3-47 works quite well.

## Save Modes

In ore Spark, saving RDDs always requires that the target directory does not exist, which can make appending to existing tables challenging. With Spark SQL, you can specify the desired behavior when writing out to a path that may already have data. The default behavior is SaveMode.ErrorIfExists; matching the behavior of RDDs, Spark will throw an exception if the target already exists. The different save modes and their habaviors are listed in Table 3-11. Example 3-48 illustrates how to confiture an alternative save mode.

## Partitions

Partition data is an important part of Spark SQL since it 